# Data Loading & Preprocessing Documentation

This notebook demonstrates how to use the data loading, preprocessing, and grouping utilities provided by the package to prepare a health dataset for intersectional fairness analysis.

The data-loading and preprocessing pipeline consists of:

- loading a tabular health dataset,
- applying fairness-oriented preprocessing,
- preparing model-ready features,
- constructing an evaluation DataFrame aligned with model predictions and true outcomes.

## Modules used in this notebook

`fairness.data`
- `load_heart_csv` - a sample data loading utility for the UCI Heart Disease dataset


`fairness.preprocess`
- `add_age_group` - add a fairness-oriented feature (eg. age bins)
- `map_binary_column` - map categorical variables to numeric form
- `apply_transforms` - sequential application of preprocessing steps
- `preprocess_tabular` - conversion to numeric features
- `make_train_test_split` - reproducible train/test partitionin


`fairness.groups`
- `make_eval_df` - constructs an evaluation DataFrame containing
intersectional group labels aligned with model predictions and true labels

## Goal

The purpose of this pipeline is to produce aligned evaluation inputs for fairness analysis.

Rather than returning multiple separate lists, 'make_eval-df' outputs a single DataFrame in which each row corresponds to one test-set individual.

For each test individual i, the evaluation DataFrame contains:

- subject_label[i] - the protected (intersectional) group label
- y_pred[i] - the model’s prediction
- y_true[i] - the true outcome

These aligned outputs can then be passed directly to group-based fairness metrics such as accuracy differences or differential fairness.

> **Note:**
> Model training is included in this notebook only to generate predictions (y_pred) for demonstration purposes.
> The data loading, preprocessing, and grouping modules are model-agnostic and can be used with any classifier that produces predictions aligned to the test set.


In [1]:
from fairness.data import load_heart_csv
from fairness.preprocess import add_age_group, map_binary_column, apply_transforms, preprocess_tabular, make_train_test_split
from fairness.groups import make_eval_df

import pandas as pd
import numpy as np

## Load the dataset

This demo uses the `heart.csv` file. 

Heart failure is a common event caused by cardiovascular diseases, and this dataset contains 11 features that can be used to predict possible heart failure.


In [2]:
from pathlib import Path

ROOT = Path.cwd().parent if Path.cwd().name == "examples" else Path.cwd()
DATA_PATH = ROOT / "data" / "heart.csv"

df_raw = load_heart_csv(DATA_PATH)
df_raw.head()


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


## Fairness-oriented preprocessing

Continuous protected attributes (like age) are binned into a small number of categories
to produce interpretable groups and avoid tiny subgroup sample sizes.

There is an optional mapping for a binary protected attribute (e.g. `Sex` from `"M"/"F"` to `1/0`),
depending on how the dataset encodes it.


In [3]:
# Add a protected attribute for fairness analysis 
df_fair = add_age_group(df_raw, age_col="Age", new_col="age_group", bins=(0, 55, 120), labels=("young", "older"))

# map binary/categorical encodings if needed (if dataset has M/F)
if "Sex" in df_fair.columns and df_fair["Sex"].dtype == object:
    df_fair = map_binary_column(df_fair, col="Sex", mapping={"M": 1, "F": 0})

df_fair[["Age", "age_group", "Sex"]].head()

,Age,age_group,Sex
0,40,young,1
1,49,young,0
2,37,young,1
3,48,young,0
4,54,young,1


### Using `apply_transforms`

`apply_transforms` allows multiple `DataFrame -> DataFrame` operations to be chained togther. 


In [4]:
df_fair2 = apply_transforms(
    df_raw,
    transforms=[
        lambda d: add_age_group(d, age_col="Age", new_col="age_group"),
        lambda d: map_binary_column(d, col="Sex", mapping={"M": 1, "F": 0}),
    ],
)

df_fair2[["Age", "age_group","Sex"]].head()

,Age,age_group,Sex
0,40,young,1
1,49,young,0
2,37,young,1
3,48,young,0
4,54,young,1


## Model-oriented preprocessing

In the raw dataset, variables are represented using a mixture of numeric and categorical encodings, reflecting how the data were originally defined and collected.

Binary clinical indicators such as `FastingBS (0, 1)` are passed through unchanged.

Variables that represent categorical concepts with 2 or ,ore possible values, such as `ChestPainType (TA, ATA, NAP, ASY)` are converted into nuermic features using one-hot encoding. This creates binary indicator columns that take value `True` if the category applies to the individual, else `False`.

This allows interpretation by machine learning models.

Some protected characteristics, such as sex, may be clinically relevant predictors
and are therefore retained in the model inputs. Derived protected attributes used
only for fairness analysis (e.g. `age_group`) are excluded. 

In [5]:
df_model = preprocess_tabular(df_fair, drop_cols=("age_group",))
df_model.head()

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,1,140,289,0,172,0.0,0,True,False,False,True,False,False,False,True
1,49,0,160,180,0,156,1.0,1,False,True,False,True,False,False,True,False
2,37,1,130,283,0,98,0.0,0,True,False,False,False,True,False,False,True
3,48,0,138,214,0,108,1.5,1,False,False,False,True,False,True,True,False
4,54,1,150,195,0,122,0.0,0,False,True,False,True,False,False,False,True


## Train/test split 

`make_train_test_split` returns a `SplitData` container:
- `X_train`, `X_test`
- `y_train`, `y_test`

Derived protected attributes (e.g. `age_group`) are dropped from the model features for training.


In [6]:
split = make_train_test_split(
    df_model,
    target_col="HeartDisease",
    test_size=0.3,
    random_state=42,
    stratify=True,
)

split.X_train.shape, split.X_test.shape, split.y_train.shape, split.y_test.shape

((642, 15), (276, 15), (642,), (276,))

## Train a model to generate `y_pred`

This step is outside of the toolkit's data loading and processing modules.
It is included here to show how `y_pred` can be 
produced for fairness metrics.


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


model = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000))
])

model.fit(split.X_train, split.y_train)
y_pred = model.predict(split.X_test)

print("\nClassification report:")
print(classification_report(split.y_test, y_pred))




Classification report:
              precision    recall  f1-score   support

           0       0.90      0.84      0.87       123
           1       0.88      0.92      0.90       153

    accuracy                           0.88       276
   macro avg       0.89      0.88      0.88       276
weighted avg       0.88      0.88      0.88       276



The table abvove simmarises the predictive performance of the trained classifier on the test set, using standard classification metrics.

Each row corresponds to one outcome class:
- Class 0: no heart disease
- Class 1: heart disease present


Accuracy is the proportion of all test-set predictions that are correct An accuracy of 0.88 means the model predicts the correct outcome for 88% of patients in the test set. In this example, the classifier achieves good accuracy, providing a suitable baseline for fairness analysis.

## Make an evaulation DataFrame

The `eval_df` DataFrame object can be passed to fairness metrics.

A list of protected attributes are passed to the function, allowing the identification of the different intersectional groups.

Each row corresponds to one individual in the test set, and the columns are:

- subject_label: the individual’s intersectional protected group
- y_pred: the model’s prediction
- y_true: the true outcome

Because all three columns are aligned row-by-row, fairness metrics can safely compare predictions and outcomes across protected groups.

In [8]:
df_test = df_fair.loc[split.X_test.index]

eval_df = make_eval_df(
    df_test=df_test,
    protected=["Sex","age_group",],
    y_pred=y_pred,
    y_true=split.y_test.to_numpy(),
)

eval_df.head(5)

,subject_label,y_pred,y_true
351,Sex=1|age_group=young,1,1
596,Sex=1|age_group=older,1,1
491,Sex=1|age_group=older,1,1
794,Sex=1|age_group=young,0,0
544,Sex=0|age_group=older,0,0


## The Evaluation dataframe can be passed to metric functions

It containes the inputs expected by group-based fairness
metrics. No further indexing or filtering is required.

```text
(test individual i)
 ├─ eval_df.subject_label[i]  - protected group
 ├─ eval_df.y_pred[i]         -  model prediction
 └─ eval_df.y_true[i]         - true outcome
```

In [9]:
subject_labels = eval_df["subject_label"].tolist()
predictions = eval_df["y_pred"].tolist()
true_statuses = eval_df["y_true"].tolist()

## Example usage (max intersectional accuracy difference)


In [10]:
from fairness.adapters import unpack_eval_df, make_subject_labels_dict
from fairness.metrics import max_intersect_acc_diff

subject_labels, predictions, true_statuses = unpack_eval_df(eval_df)

subject_labels_dict = make_subject_labels_dict(
    df_test, protected_cols=["Sex", "age_group"]
)

maxdiff = max_intersect_acc_diff(subject_labels_dict, predictions, true_statuses)
print(maxdiff)


0.06928294573643412


---

## Quick pipeline for internal / development use only (not part of API)

For convenience, a whole pipeline function is included at `utils/pipeline.py`. Using the code below, this can be called to get:
- a fitted model
- `y_test` (true values)
- `y_pred` (predicted values)
- `eval_df` (evaluation dataframe)

It's designed for our convenience while developing the package and is not intedned to be part of the final package.

In [11]:
from fairness.preprocess import add_age_group, map_binary_column
from fairness.demo.pipeline import run_demo_pipeline

result = run_demo_pipeline(
    csv_path="../data/heart.csv",
    target_col="HeartDisease",
    protected_cols=["Sex", "age_group"],
    fairness_transforms=[
        lambda d: add_age_group(
            d,
            age_col="Age",
            new_col="age_group",
            bins=(0, 55, 120),
            labels=("young", "older"),
        ),
        lambda d: map_binary_column(d, col="Sex", mapping={"M": 1, "F": 0}),
    ],
    drop_from_X=("age_group",),
)

y_pred = result.y_pred
y_pred_clean = y_pred.astype(int).tolist()

y_test = result.split.y_test
eval_df = result.eval_df  



In [12]:
print("\nEvaluation DataFrame (first 5 rows):")
print(eval_df.head(5))


Evaluation DataFrame (first 5 rows):
             subject_label  y_pred  y_true
351  Sex=1|age_group=young       1       1
596  Sex=1|age_group=older       1       1
491  Sex=1|age_group=older       1       1
794  Sex=1|age_group=young       0       0
544  Sex=0|age_group=older       0       0
